## Start

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
! pip install transformers -q
! pip install dgl-cu100 -q
! pip install word2vec -q

     |████████████████████████████████| 501kB 2.8MB/s 
     |████████████████████████████████| 3.7MB 10.2MB/s 
     |████████████████████████████████| 1.0MB 39.1MB/s 
     |████████████████████████████████| 870kB 45.6MB/s 
     |████████████████████████████████| 16.2MB 200kB/s 
     |████████████████████████████████| 61kB 2.0MB/s 


In [3]:
import csv
import re
import pandas as pd
from pathlib import Path
import matplotlib.cm as cm
from fastai import *
from fastai.text import *
from fastai.callbacks import *
from fastai.metrics import *
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *
from tqdm.notebook import tqdm

import torch
import torch.optim as optim
import dgl
import word2vec

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

In [6]:
import os
os.chdir('/gdrive/My Drive/DEFINITION EXTRACTION/DEFT_Updated')
! ls

BERT_attention_kfold.ipynb	   rbert_ft
bert_cached_lm_510_deft.test.raw   roberta_cached_lm_510_deft.test.raw
bert_cached_lm_510_deft.train.raw  roberta_cached_lm_510_deft.train.raw
bert_ft				   RobertaLM_GB_kfold.ipynb
catboost_info			   roberta_tuned_test.npy
deft.test.raw			   roberta_tuned_train.npy
deft.train.raw			   runs
DualXLNet_attention_kfold.ipynb    sub
GCN_BERT_attention_kfold.ipynb	   task1_dev.csv
GCN_BERT_dependency_kfold.ipynb    task1_test.csv
GCN_BERT_final.ipynb		   task1_train.csv
gcn_dep.ipynb			   train_lm.py
gcn.ipynb			   vocab.txt
gcn_model.pkl			   WCL_cased_symbols.csv
glove.6B.200d.vec.txt		   XLNET_attention_kfold.ipynb
models				   XLNET_GCN_attention_kfold.ipynb


In [0]:
model_name="bert-base-cased"
# model_name="roberta-base"
# model_name="xlnet-base-cased"

In [0]:
TEXT_COL = 'Sentence'
Y_COL = 'Label'

In [0]:
import re
def multi_replace(chars_to_replace, char_to_place, from_string):
    for char in chars_to_replace:
        from_string = from_string.replace(char,char_to_place)
    return from_string

def preprocessing(sentence):
    remove_punct = multi_replace(',“”’.-{}[]()=+_!@#$%^&*<>?/|\\~`:;"','',sentence)
    return remove_punct

def load_data(file):
    df = pd.read_csv(file)
    # removing initial numbers
    df[TEXT_COL] = df[TEXT_COL].apply(lambda x : re.findall('^\s*\d*\s*\.?\s*(.*)',x)[0])
    # preprocessing
    # df[TEXT_COL] = df[TEXT_COL].apply(preprocessing)
    return df


In [10]:
train_df = load_data('task1_train.csv')
train_df.head(2)

,Subject,Sentence,Label
0,biology,Science includes such diverse fields as astron...,0
1,biology,"However , those fields of science related to t...",1


In [11]:
dev_df = load_data('task1_dev.csv')
dev_df.head(2)

,Subject,Sentence,Label
0,biology,It becomes clear from this definition that the...,0
1,biology,The scientific method is a method of research ...,1


In [12]:
test_df = load_data('task1_test.csv')
test_df.head(2)

,Subject,Sentence,Label\n
0,biology,Living things are highly organized and structu...,NaN
1,biology,The atom is the smallest and most fundamental ...,NaN


## GCN

In [13]:
from keras.preprocessing.sequence import pad_sequences

class DataHelper(object):
    def __init__(self, df, mode='train', vocab=None):
        self.mode = mode
        
        self.df = df
        if self.mode=='train':
          content, label = self.get_content()
          self.label = self.label_to_onehot(label)

        else:
          content = self.get_content()
        
        if vocab is None:
            self.vocab = []
            try:
                print(f'Vocab file found')
                self.get_vocab()
            except FileNotFoundError:
                self.build_vocab(content, min_count=5)
        else:
            self.vocab = vocab

        self.d = dict(zip(self.vocab, range(len(self.vocab))))
        self.rev_d = { v:k for k,v in self.d.items()}
        self.content = [list(map(lambda x: self.word2id(x), doc.split(' '))) for doc in content]
        self.lengths = [[len(s)] for s in self.content]
        self.content = pad_sequences(self.content, maxlen=64, dtype="long", truncating="post", padding="post")


    def label_to_onehot(self, label_str):
      return np.array(self.df[Y_COL])
        
    def get_content(self):
        if self.mode=='train':
          return list(self.df[TEXT_COL]), list(self.df[Y_COL])
        else:
          return list(self.df[TEXT_COL])

    def word2id(self, word):
        try:
            result = self.d[word]
        except KeyError:
            result = self.d['UNK']
        return result

    def get_vocab(self):
        with open('vocab-5.txt') as f:
            vocab = f.read()
            self.vocab = vocab.split('\n')

    def build_vocab(self, content, min_count=10):
        vocab = []
        for c in content:
            words = c.split(' ')
            for word in words:
                if word not in vocab:
                    vocab.append(word)
        freq = dict(zip(vocab, [0 for i in range(len(vocab))]))
        for c in content:
            words = c.split(' ')
            for word in words:
                freq[word] += 1
        results = []
        for word in freq.keys():
            if freq[word] < min_count:
                continue
            else:
                results.append(word)
        results.insert(0, 'UNK')
        with open('vocab.txt', 'w') as f:
            f.write('\n'.join(results))
        self.vocab = results

    def count_word_freq(self, content):
        freq = dict(zip(self.vocab, [0 for i in range(len(self.vocab))]))
        for c in content:
            words = c.split(' ')
            for word in words:
                freq[word] += 1
        with open('freq.csv', 'w') as f:
            writer = csv.writer(f)
            results = list(zip(freq.keys(), freq.values()))
            writer.writerows(results)

    def batch_iter(self, batch_size, num_epoch):
        for i in range(num_epoch):
            num_per_epoch = int(len(self.content) / batch_size)
            for batch_id in range(num_per_epoch):
                start = batch_id * batch_size
                end = min((batch_id + 1) * batch_size, len(self.content))
                content = self.content[start:end]
                label = self.label[start:end]
                yield (content), torch.LongTensor(label).cuda(), i


Using TensorFlow backend.


In [14]:
train_data_helper = DataHelper(train_df, mode='train')
dev_data_helper = DataHelper(dev_df, mode='train')
test_data_helper = DataHelper(test_df, mode='test')

Vocab file found
Vocab file found
Vocab file found


## Transformer

In [15]:
 %tensorflow_version 1.x 
from transformers import *
tokenizer = BertTokenizer.from_pretrained(model_name, output_attentions=True)

## Preprocessing

In [0]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 64

def tokenise_pad(df):
  X = [tokenizer.encode(x, add_special_tokens=True) for x in df[TEXT_COL]]
  X = pad_sequences(X, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  return X


X_train = tokenise_pad(train_df)
X_dev = tokenise_pad(dev_df)
X_test = tokenise_pad(test_df)

In [0]:
def return_attention_masks(X):
  attention_masks = []
  for seq in X:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
  attention_masks = np.array(attention_masks)  
  return attention_masks

In [0]:
X_train_masks = return_attention_masks(X_train)
X_dev_masks = return_attention_masks(X_dev)
X_test_masks = return_attention_masks(X_test)


In [0]:
y_train = np.array(train_df[Y_COL])
y_dev = np.array(dev_df[Y_COL])

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

## Models

In [0]:
def cal_PMI(helper, window_size=20):
    content, _ = helper.get_content()
    pair_count_matrix = np.zeros((len(helper.vocab), len(helper.vocab)), dtype=int)
    word_count =np.zeros(len(helper.vocab), dtype=int)

    for sentence in tqdm(content):
        sentence = sentence.split(' ')
        for i, word in enumerate(sentence):
            try:
                word_count[helper.d[word]] += 1
            except KeyError:
                continue
            start_index = max(0, i - window_size)
            end_index = min(len(sentence), i + window_size)
            for j in range(start_index, end_index):
                if i == j:
                    continue
                else:
                    target_word = sentence[j]
                    try:
                        pair_count_matrix[helper.d[word], helper.d[target_word]] += 1
                    except KeyError:
                        continue

    total_count = np.sum(word_count)
    word_count = word_count / total_count
    pair_count_matrix = pair_count_matrix / total_count

    pmi_matrix = np.zeros((len(helper.vocab), len(helper.vocab)), dtype=float)
    for i in range(len(helper.vocab)):
        for j in range(len(helper.vocab)):
            pmi_matrix[i, j] = np.log(
                pair_count_matrix[i, j] / (word_count[i] * word_count[j])
            )

    pmi_matrix = np.nan_to_num(pmi_matrix)

    pmi_matrix = np.maximum(pmi_matrix, 0.0)

    edges_weights = [0.0]
    count = 1
    edges_mappings = np.zeros((len(helper.vocab), len(helper.vocab)), dtype=int)
    for i in range(len(helper.vocab)):
        for j in range(len(helper.vocab)):
            if pmi_matrix[i, j] != 0:
                edges_weights.append(pmi_matrix[i, j])
                edges_mappings[i, j] = count
                count += 1

    edges_weights = np.array(edges_weights)

    edges_weights = edges_weights.reshape(-1, 1)
    # print(edges_weights.shape)
    edges_weights = torch.Tensor(edges_weights)

    return edges_weights, edges_mappings, count


In [0]:
def gcn_msg(edge):
    return {'m': edge.src['h'], 'w': edge.data['w']}


def gcn_reduce(node):
    w = node.mailbox['w']

    new_hidden = torch.mul(w, node.mailbox['m'])

    new_hidden,_ = torch.max(new_hidden, 1)

    node_eta = torch.sigmoid(node.data['eta'])
    # node_eta = F.leaky_relu(node.data['eta'])

    # new_hidden = node_eta * node.data['h'] + (1 - node_eta) * new_hidden
    # print(new_hidden.shape)

    return {'h': new_hidden}


class GCNModel(torch.nn.Module):
    def __init__(self,
                 hidden_size_node,
                 vocab,
                 n_gram,
                 edges_num,
                 edges_matrix,
                 max_length=350,
                 trainable_edges=True,
                 pmi=None,
                 cuda=True
                 ):
        super(GCNModel, self).__init__()

        self.is_cuda = cuda
        self.vocab = vocab
        
        self.seq_edge_w = torch.nn.Embedding(edges_num, 1)
        print(edges_num)
        print(pmi.shape)

        self.node_hidden = torch.nn.Embedding(len(vocab), hidden_size_node)
        
        self.seq_edge_w = torch.nn.Embedding.from_pretrained(pmi, freeze=True)
            
        self.edges_num = edges_num
        if trainable_edges:
            self.seq_edge_w = torch.nn.Embedding.from_pretrained(torch.ones(edges_num, 1), freeze=False)
        else:
            self.seq_edge_w = torch.nn.Embedding.from_pretrained(pmi, freeze=True)

        self.hidden_size_node = hidden_size_node

        self.node_hidden.weight.data.copy_(torch.tensor(self.load_word2vec('glove.6B.200d.vec.txt')))
        self.node_hidden.weight.requires_grad = True

        self.len_vocab = len(vocab)

        self.ngram = n_gram
        self.d = dict(zip(self.vocab, range(len(self.vocab))))
        self.max_length = max_length
        self.edges_matrix = edges_matrix

        self.dropout = torch.nn.Dropout(0.5)
        self.activation = torch.nn.ReLU()

        

    def word2id(self, word):
        try:
            result = self.d[word]
        except KeyError:
            result = self.d['UNK']

        return result

    def load_word2vec(self, word2vec_file):
        model = word2vec.load(word2vec_file)
        embedding_matrix = []
        for word in self.vocab:
            try:
                embedding_matrix.append(model[word])
            except KeyError:
                # print(word)
                embedding_matrix.append(model['the'])
        embedding_matrix = np.array(embedding_matrix)
        return embedding_matrix

    

    def add_seq_edges(self, doc_ids: list, old_to_new: dict):
        edges = []
        old_edge_id = []
        for index, src_word_old in enumerate(doc_ids):
            src = old_to_new[int(src_word_old.item())]
            for i in range(max(0, index - self.ngram), min(index + self.ngram + 1, len(doc_ids))):
                dst_word_old = doc_ids[i].item()
                dst = old_to_new[dst_word_old]

                # - first connect the new sub_graph
                edges.append([src, dst])
                # - then get the hidden from parent_graph
                old_edge_id.append(self.edges_matrix[src_word_old, dst_word_old])

            # self circle
            edges.append([src, src])
            old_edge_id.append(self.edges_matrix[src_word_old, src_word_old])
        return edges, old_edge_id

    def seq_to_graph(self, doc_ids: list, doc_length) -> dgl.DGLGraph():
        doc_ids = doc_ids[0:doc_length]
        if len(doc_ids) > self.max_length:
            doc_ids = doc_ids[:self.max_length]
        local_vocab = set(doc_ids)

        old_to_new = {}
        for i,j in enumerate(local_vocab):
          old_to_new[j.item()] = i
        # old_to_new = dict(zip(local_vocab, range(len(local_vocab))))

        # print(old_to_new)

        if self.is_cuda:
            local_vocab = torch.tensor(list(local_vocab)).cuda()
        else:
            local_vocab = torch.tensor(list(local_vocab))

        sub_graph = dgl.DGLGraph()

        sub_graph.add_nodes(len(local_vocab))
        local_node_hidden = self.node_hidden(local_vocab)

        sub_graph.ndata['h'] = local_node_hidden

        seq_edges, seq_old_edges_id = self.add_seq_edges(doc_ids, old_to_new)

        edges, old_edge_id = [], []
        
        edges.extend(seq_edges)

        old_edge_id.extend(seq_old_edges_id)

        if self.is_cuda:
            old_edge_id = torch.LongTensor(old_edge_id).cuda()
        else:
            old_edge_id = torch.LongTensor(old_edge_id)

        srcs, dsts = zip(*edges)
        sub_graph.add_edges(srcs, dsts)
        try:
            seq_edges_w = self.seq_edge_w(old_edge_id)
        except RuntimeError:
            print(old_edge_id)
        sub_graph.edata['w'] = seq_edges_w

        return sub_graph

    def forward(self, doc_ids, doc_lengths):
        sub_graphs = [self.seq_to_graph(doc, length) for doc, length in zip(doc_ids, doc_lengths)]

        batch_graph = dgl.batch(sub_graphs)

        batch_graph.update_all(
            message_func=dgl.function.src_mul_edge('h', 'w', 'weighted_message'),
            reduce_func=dgl.function.max('weighted_message', 'h')
        )

        h1 = dgl.sum_nodes(batch_graph, feat='h')
        drop1 = self.dropout(h1)
        act1 = self.activation(drop1)
        # l = self.Linear(act1)
        return act1


In [0]:
class BertFeature(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.init_weights()
        self.features = config.hidden_size

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):

            bert_outputs = self.bert(
              input_ids,
              attention_mask=attention_mask,
              token_type_ids=token_type_ids,
              position_ids=position_ids,
              head_mask=head_mask,
              inputs_embeds=inputs_embeds,
            )
            pooled_output = bert_outputs[1]

            return pooled_output

## Main Model

In [0]:
class BertGCN(nn.Module):
  def __init__(self, hidden, vocabulary, n_grams, edges_mappings, edges_weights, count, num_classes, dropout_prob):
    super(BertGCN, self).__init__()
    self.bert = BertFeature.from_pretrained('bert-base-cased', output_attentions=False)
    self.gcn = GCNModel(hidden_size_node = hidden,
                            vocab = vocabulary,
                            n_gram = n_grams,
                            edges_matrix = edges_mappings,
                            edges_num = count,
                            trainable_edges = True, 
                            pmi = edges_weights, 
                            cuda = True
                            )
    self.num_classes = num_classes
    self.dropout = nn.Dropout(dropout_prob)
    self.bn1 = nn.BatchNorm1d(self.bert.features)
    self.relu1 = torch.nn.ReLU()
    self.relu2 = torch.nn.ReLU()
    
    self.bn2 = nn.BatchNorm1d(hidden)
    
    self.classifier = nn.Linear(self.bert.features + hidden, self.num_classes)

  def forward(self, input_ids, attention_mask, input_gcn, sent_len, labels=None):
    bert_out = self.bert(input_ids, attention_mask)
    gcn_out = self.gcn(input_gcn, sent_len)

    # return bert_out, gcn_out

    # bert_out = self.bn1(bert_out)
    # gcn_out = self.bn2(gcn_out)
    
    # bert_out = self.relu1(bert_out)
    # gcn_out = self.relu2(gcn_out)

    out = torch.cat((bert_out, gcn_out), 1)
    # pooled_output = self.dropout(out)
    logits = self.classifier(out)

    outputs = (logits,)

    if labels is not None:
        if self.num_classes == 1:
            loss_fct = MSELoss()
            loss = loss_fct(logits.view(-1), labels.view(-1))
        else:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_classes), labels.view(-1))
        outputs = (loss,) + outputs
    return outputs

## DEBUG

In [25]:
edges_weights, edges_mappings, count = cal_PMI(train_data_helper, 5)

In [0]:
# model = BertGCN(hidden = 200,
#                 vocabulary = train_data_helper.vocab,
#                 n_grams = 5,
#                 edges_mappings = edges_mappings,
#                 edges_weights = edges_weights, 
#                 count = count, 
#                 num_classes = 2, 
#                 dropout_prob = 0.3
#                 )

# Training

In [26]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
device

device(type='cuda', index=0)

In [0]:
from sklearn.utils.extmath import softmax
from sklearn.metrics import classification_report, f1_score
BATCH_SIZE = 32


def flat_accuracy(preds, labels):
    preds = softmax(preds)
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def flat_f1(preds, labels):
    preds = softmax(preds)
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat)


In [0]:
X_train_gcn = train_data_helper.content
X_train_len = train_data_helper.lengths

X_dev_gcn = dev_data_helper.content
X_dev_len = dev_data_helper.lengths

X_test_gcn = test_data_helper.content
X_test_len = test_data_helper.lengths

In [0]:
X_train_gcn = np.array(X_train_gcn)
X_dev_gcn = np.array(X_dev_gcn)
X_test_gcn = np.array(X_test_gcn)

X_train_len = np.array(X_train_len)
X_dev_len = np.array(X_dev_len)
X_test_len = np.array(X_test_len)

In [0]:
X_train = torch.tensor(X_train)
X_train_masks = torch.tensor(X_train_masks)
X_train_gcn = torch.tensor(X_train_gcn)
X_train_len = torch.tensor(X_train_len)
y_train = torch.tensor(y_train)

train_data = TensorDataset(X_train, X_train_masks, X_train_gcn, X_train_len, y_train)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

In [0]:
X_dev = torch.tensor(X_dev)
X_dev_masks = torch.tensor(X_dev_masks)
X_dev_gcn = torch.tensor(X_dev_gcn)
X_dev_len = torch.tensor(X_dev_len)
y_dev = torch.tensor(y_dev)


dev_data = TensorDataset(X_dev, X_dev_masks, X_dev_gcn, X_dev_len, y_dev)
dev_sampler = SequentialSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=BATCH_SIZE)

In [0]:
X_test = torch.tensor(X_test)
X_test_masks = torch.tensor(X_test_masks)
X_test_gcn = torch.tensor(X_test_gcn)
X_test_len = torch.tensor(X_test_len)

test_data = TensorDataset(X_test, X_test_masks, X_test_gcn, X_test_len)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

### Loop

In [0]:
from tqdm.notebook import tqdm
import sys
auto_model = None


del auto_model
auto_model = BertGCN(hidden = 200,
            vocabulary = train_data_helper.vocab,
            n_grams = 5,
            edges_mappings = edges_mappings,
            edges_weights = edges_weights, 
            count = count, 
            num_classes = 2, 
            dropout_prob = 0.3
            )


auto_model.to(device)

# Optimizer
num_total_steps = 1000
num_warmup_steps = 100
lr = 5e-6
param_optimizer = list(auto_model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
  {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
  'weight_decay_rate': 0.01},
  {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
  'weight_decay_rate': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=lr)

best_loss = None

auto_model.train()  
tr_loss = 0
nb_tr_examples, nb_tr_steps = 0, 0
n_epochs = 5

# Training Loop
for epoch in (range(n_epochs)):  
  auto_model.train()
  for step, batch in (enumerate(train_dataloader)):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_input_gcn ,b_len, b_labels = batch
    optimizer.zero_grad()
    outputs = auto_model(b_input_ids, 
                          b_input_mask, 
                          b_input_gcn, 
                          b_len, 
                          b_labels)

    loss, logits = outputs[:2]
    loss.backward()
    optimizer.step()
    
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1


  # Validation Loop
  auto_model.eval()
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  yt, yp = [], []
  for batch in dev_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_input_gcn ,b_len, b_labels = batch

    with torch.no_grad():
      outputs = auto_model(b_input_ids, 
                            attention_mask=b_input_mask,
                            input_gcn = b_input_gcn,
                            sent_len = b_len,
                            labels= b_labels)        
      loss, logits = outputs[:2]
    logits = logits.detach().cpu().numpy()

    preds = softmax(logits)
    pred_ids = np.argmax(preds, axis=1).flatten()

    label_ids = b_labels.to('cpu').numpy()

    eval_loss += loss.item()
    eval_accuracy += flat_accuracy(logits, label_ids)

    yt = yt + label_ids.tolist()
    yp = yp + pred_ids.tolist()

    nb_eval_steps +=1
        
  print("Epoch {} | Train loss: {} | Validation Loss: {} ".format(epoch,
                                                                    tr_loss/nb_tr_steps,
                                                                    eval_loss/nb_eval_steps, 
                                                                  ))
  if best_loss == None or best_loss > eval_loss/nb_eval_steps:
    best_loss = eval_loss/nb_eval_steps
    torch.save(auto_model.state_dict(), 'models/check_bertgcn.pth')
    print(f'Saving model')


print(classification_report(yt, yp))   

873034
torch.Size([873034, 1])
Epoch 0 | Train loss: 0.4278146782855848 | Validation Loss: 0.4144361753154684 
Saving model
Epoch 1 | Train loss: 0.3749028979613012 | Validation Loss: 0.3969629853963852 
Saving model
Epoch 2 | Train loss: 0.34298959613455887 | Validation Loss: 0.3737640215290917 
Saving model
Epoch 3 | Train loss: 0.3172001071980448 | Validation Loss: 0.35195665723747677 
Saving model
Epoch 4 | Train loss: 0.294219728980227 | Validation Loss: 0.3822715911600325 
              precision    recall  f1-score   support

           0       0.87      0.97      0.92       582
           1       0.91      0.69      0.78       272

    accuracy                           0.88       854
   macro avg       0.89      0.83      0.85       854
weighted avg       0.88      0.88      0.87       854



In [34]:
auto_model = BertGCN(hidden = 200,
            vocabulary = train_data_helper.vocab,
            n_grams = 5,
            edges_mappings = edges_mappings,
            edges_weights = edges_weights, 
            count = count, 
            num_classes = 2, 
            dropout_prob = 0.3
            )



auto_model.load_state_dict(torch.load('models/check_bertgcn.pth'))
auto_model.to(device)
auto_model.eval()
print('Loading best model')

yt, yp, p = [], [], []
for batch in dev_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_input_gcn ,b_len, b_labels = batch

  with torch.no_grad():
    outputs = auto_model(b_input_ids, 
                            attention_mask=b_input_mask,
                            input_gcn = b_input_gcn,
                            sent_len = b_len,
                            labels= b_labels)          
    loss, logits = outputs[:2]
  logits = logits.detach().cpu().numpy()
  preds = softmax(logits)

  pred_ids = np.argmax(preds, axis=1).flatten()
  label_ids = b_labels.to('cpu').numpy()

  p = p + preds[:,1].tolist()
  yt = yt + label_ids.tolist()
  yp = yp + pred_ids.tolist()

873034
torch.Size([873034, 1])
Loading best model


In [35]:
dev_df['Probability'] = p
dev_df['Predicted'] = yp
dev_df['Loss'] = -dev_df['Label']*np.log2(dev_df['Probability']) - (1-dev_df['Label'])*np.log2(1-dev_df['Probability'])
dev_df = dev_df.sort_values(by='Loss', axis=0, ascending=False)
dev_df.head()

,Subject,Sentence,Label,Probability,Predicted,Loss
653,government,"”United States v. Miller , 307 U.S. 174 ( 1939...",1,0.008916,0,6.809331
40,biology,"Pathogens include bacteria , protists , fungi ...",1,0.009523,0,6.714331
570,sociology,While pastoral and horticultural societies use...,1,0.010113,0,6.627664
702,government,"Starting in New York in 1790 , the early Supre...",1,0.010170,0,6.619555
706,government,The practice of judicial review enabled the la...,1,0.012451,0,6.327596


In [36]:
c = 10
for i in dev_df['Sentence']:
  print(i)
  c-=1
  if c==0:
    break

”United States v. Miller , 307 U.S. 174 ( 1939 ) .
Pathogens include bacteria , protists , fungi and other infectious organisms .
While pastoral and horticultural societies used small , temporary tools such as digging sticks or hoes , agricultural societies relied on permanent tools for survival .
Starting in New York in 1790 , the early Supreme Court focused on establishing its rules and procedures and perhaps trying to carve its place as the new government ’s third branch .
The practice of judicial review enabled the law ’s critics to exercise this opportunity , even though their hopes were ultimately dashed when , by a narrow 5 – 4 margin , the Supreme Court upheld the health care law as a constitutional extension of Congress ’s power to tax .
Radical Whigs favored broadening the popular participation in political life and pushed for democracy .
Perhaps the greatest real estate deal in American history , the Louisiana Purchase greatly enhanced the Jeffersonian vision of the United S

In [39]:
dev_df.loc[706, ['Sentence', 'Loss']]

Sentence    The practice of judicial review enabled the la...
Loss                                                   6.3276
Name: 706, dtype: object

In [42]:
from sklearn.metrics import confusion_matrix, f1_score
confusion_matrix(yt, yp), f1_score(yt, yp)

(array([[565,  17],
        [ 87, 185]]), 0.7805907172995781)

# Test Predictions

In [41]:
test_preds = []
auto_model.load_state_dict(torch.load('models/check_bertgcn.pth'))
auto_model.eval()
print('Loading best model')

for batch in test_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_input_gcn ,b_len = batch

  with torch.no_grad():
    outputs = auto_model(b_input_ids, 
                          b_input_mask,
                          b_input_gcn,
                          b_len
                        )        
    logits = outputs[0]
  logits = logits.detach().cpu().numpy()
  preds = softmax(logits)[:, 1]        
  test_preds = test_preds + preds.tolist()



Loading best model


In [0]:
test_df['Probability'] = test_preds
test_df['Probability'].describe()

count    859.000000
mean       0.231181
std        0.338427
min        0.005361
25%        0.010326
50%        0.029999
75%        0.328995
max        0.980175
Name: Probability, dtype: float64

In [0]:
test_df.to_csv('sub/submission_bert_gcn_final.csv', index=False)